In [1]:
from lucam import Lucam

In [286]:
cam = Lucam()
capture = cam.TakeSnapshot()
cam.SaveImage(capture, 'test.png')
cam.CameraClose()

In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2

um = 1e-6
mm = 1e-3

def ASM(image, lam, dx, z):
    k = 2 * np.pi / lam
    N = image.shape[0]
    df = 1 / N / dx
    
    fx = np.arange(-N//2, N//2) * df
    fy = fx

    Fx, Fy = np.meshgrid(fx, fy)
    
    kz_squared = k**2 - (2 * np.pi * Fx)**2 - (2 * np.pi * Fy)**2
    kz_squared[kz_squared<0] = 0
    kz = np.sqrt(kz_squared)
    Hz = np.exp(1j * kz * z)
    
    fimage = np.fft.fftshift(np.fft.fft2(image))
    fimage = fimage * Hz
    pimage = np.fft.ifft2(np.fft.ifftshift(fimage))
    
    return pimage

In [6]:
from PIL import Image
import numpy as np
import cv2
import torch
from lucam import Lucam
import napari

um = 1e-6
mm = 1e-3

def ASM_cuda(image, lam, dx, z):
    k = 2 * np.pi / lam
    N = image.shape[0]
    df = 1 / N / dx
    
    fx = np.arange(-N//2, N//2) * df
    fy = fx

    Fx, Fy = np.meshgrid(fx, fy)
    
    kz_squared = k**2 - (2 * np.pi * Fx)**2 - (2 * np.pi * Fy)**2
    kz_squared[kz_squared<0] = 0
    kz = np.sqrt(kz_squared)
    Hz = np.exp(1j * kz * z)

    Hz = torch.from_numpy(Hz).cuda()
    
    fimage = torch.fft.fftshift(torch.fft.fft2(image))
    fimage = fimage * Hz
    pimage = torch.fft.ifft2(torch.fft.ifftshift(fimage))
    
    return pimage

In [177]:
def fresnel(image, lam, dx, z):

    N = 600
    L = N * dx

    x = np.linspace(-L/2, L/2, N)
    y = x
    X, Y = np.meshgrid(x, y)

    k = 2 * np.pi / lam
    coeff = np.exp(1j * k * z) / (1j * lam * z)
    kernel = coeff * np.exp(1j * k / 2 / z * (X**2 + Y**2))
    transfer = np.fft.fftshift(np.fft.fft2(kernel))

    f_image = np.fft.fftshift(np.fft.fft2(image))
    f_image = f_image * transfer
    image = np.fft.ifft2(np.fft.ifftshift(f_image))
    image = np.fft.fftshift(image) # Don't know why do this but this should be exist
    return image

In [3]:
def fresnel_cuda(image, lam, dx, z):

    N = 600
    L = N * dx

    x = np.linspace(-L/2, L/2, N)
    y = x
    X, Y = np.meshgrid(x, y)

    k = 2 * np.pi / lam
    coeff = np.exp(1j * k * z) / (1j * lam * z)
    kernel = coeff * np.exp(1j * k / 2 / z * (X**2 + Y**2))
    transfer = np.fft.fftshift(np.fft.fft2(kernel))

    transfer = torch.from_numpy(transfer).cuda()
    
    f_image = torch.fft.fftshift(torch.fft.fft2(image))
    f_image = f_image * transfer
    image = torch.fft.ifft2(torch.fft.ifftshift(f_image))
    image = torch.fft.fftshift(image) # Don't know why do this but this should be exist
    return image

In [9]:
image = Image.open('rikka_edge.png').convert('L')
image = np.array(image)
image = cv2.resize(image, dsize=(600,600))
N = 600
image.shape

(600, 600)

In [187]:
image = 255 - image

<h1> Gerchbach-Saxton </h1>

In [188]:
slm_phase = np.ones((N,N)) * np.pi
slm_field = np.exp(1j * slm_phase)

target = image.copy()

In [175]:
from tqdm import tqdm

lam = 0.532 * um
dx = 12.5 * um
z = 200 * mm

errors = []

for i in tqdm(range(200)):
    field_at_target_plane = fresnel(slm_field, lam, dx, z)
    desired_field_at_target = np.sqrt(image/np.max(image)) * np.exp(1j * np.angle(field_at_target_plane))
    field_at_slm_plane = fresnel(desired_field_at_target, lam, dx, -z)
    slm_field = np.exp(1j * np.angle(field_at_slm_plane))
    
    if (i + 1) % 10 == 0:
        print(f"Iteration {i+1}/{200} completed.")
        # Optional: Calculate and display current reconstruction
        current_reconstruction_field = fresnel(slm_field, lam, dx, z)
        current_reconstruction_intensity = np.abs(current_reconstruction_field)**2
        
        target = image.copy()
        current = current_reconstruction_intensity.copy()
        target = target / np.max(target)
        current = current / np.max(current)
        
        error = np.sum(np.abs(target-current)**2)/N/N
        print(error)
        
        errors.append(error)
        
        

  5%|▌         | 10/200 [00:02<00:53,  3.56it/s]

Iteration 10/200 completed.
0.020070159764653792


 10%|█         | 20/200 [00:05<00:48,  3.68it/s]

Iteration 20/200 completed.
0.01995372905841479


 15%|█▌        | 30/200 [00:07<00:46,  3.65it/s]

Iteration 30/200 completed.
0.021104012842585552


 16%|█▌        | 32/200 [00:08<00:43,  3.82it/s]


KeyboardInterrupt: 

In [7]:
from tqdm import tqdm

lam = 0.532 * um
dx = 12.5 * um
z = 100 * mm

errors = []

image = torch.from_numpy(image).cuda()
slm_field = torch.from_numpy(slm_field).cuda()

for i in tqdm(range(200)):
    field_at_target_plane = fresnel_cuda(slm_field, lam, dx, z)
    desired_field_at_target = torch.sqrt(image/torch.max(image)) * torch.exp(1j * torch.angle(field_at_target_plane))
    field_at_slm_plane = fresnel_cuda(desired_field_at_target, lam, dx, -z)
    slm_field = torch.exp(1j * torch.angle(field_at_slm_plane))
    
    if (i + 1) % 10 == 0:
        print(f"Iteration {i+1}/{200} completed.")
        # Optional: Calculate and display current reconstruction
        current_reconstruction_field = fresnel_cuda(slm_field, lam, dx, z)
        current_reconstruction_intensity = torch.abs(current_reconstruction_field)**2
        
        current = current_reconstruction_intensity.detach().cpu()
        current = np.array(current)
        target = target / np.max(target)
        current = current / np.max(current)
        
        error = np.sum(np.abs(target-current)**2)/N/N
        print(error)
        
        errors.append(error)

image = image.detach().cpu()
image = np.array(image)
slm_field = slm_field.detach().cpu()
slm_field = np.array(slm_field)
        

NameError: name 'slm_field' is not defined

<h1> Hybrid Input Output </h1>

In [19]:
image = pattern.copy()

In [14]:
slm_phase = np.ones((N,N)) * np.pi
slm_field = np.exp(1j * slm_phase)
target = image.copy()

In [17]:
from tqdm import tqdm

lam = 0.532 * um
dx = 12.5 * um
z = 50 * mm

errors = []

image = torch.from_numpy(image).cuda()
slm_field = torch.from_numpy(slm_field).cuda()

for i in tqdm(range(10)):
    field_at_target_plane = ASM_cuda(slm_field, lam, dx, z)
    desired_field_at_target = torch.sqrt(image/torch.max(image)) * torch.exp(1j * torch.angle(field_at_target_plane))
    update = ASM_cuda(desired_field_at_target, lam, dx, -z)
    
    mask = torch.zeros((N, N), dtype=bool).cuda()
    mask[torch.imag(update) < 1] = True
    slm_field[mask] = update[mask]
    slm_field[~mask] = slm_field[~mask] - 0.9 * update[~mask]
    if (i + 1) % 10 == 0:
    # if i >= 0:
        print(f"Iteration {i+1}/{200} completed.")
        # Optional: Calculate and display current reconstruction
        current_reconstruction_field = ASM_cuda(slm_field, lam, dx, z)
        current_reconstruction_intensity = torch.abs(current_reconstruction_field)**2
        
        current = current_reconstruction_intensity.detach().cpu()
        current = np.array(current)
        target = target / np.max(target)
        current = current / np.max(current)
        
        error = np.sum(np.abs(target-current)**2)/N/N
        print(error)
        
        errors.append(error)

image = image.detach().cpu()
image = np.array(image)
slm_field = slm_field.detach().cpu()
slm_field = np.array(slm_field)
        

100%|██████████| 10/10 [00:00<00:00, 11.15it/s]

Iteration 10/200 completed.
2.9141701575514257e-17


In [18]:
angle = np.angle(slm_field)
angle = (angle - np.min(angle)) / (np.max(angle) - np.min(angle)) * 255
angle = angle.astype('uint8')
Image.fromarray(angle).save('test.png')

In [2]:
pattern = np.ones((600, 600)) * 255
Image.fromarray(pattern.astype('uint8')).save('test.png')